<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Hockey%20Model%20Mark%20I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Notes**
1. On-ice EV xG+/-

2. On-ice EV G+/-

3. On-ice PP G+/- above average

4. On-ice SH G+/- above average

5. GSAx

6. Individual points above average (depending on position and role)

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [2]:
#Import Even-Strength On-Ice Totals Data
skater_EV_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/EV%20On-Ice%20Totals.csv')
skater_EV_totals_raw.head()

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,57.90,57.33,56.89,...,54.34,3.16,2.22,0.06,10.82,14.68,17.36,0.94,7.59,89.76
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,54.75,53.16,53.30,53.14,...,1070.09,52.60,52.78,12.38,84.53,114.87,143.58,-0.18,10.03,90.59
2,Aaron Ekblad,20-21,FLA,D,35,630.15,57.67,54.51,53.65,53.97,...,481.39,26.51,21.58,6.70,54.32,59.26,83.07,4.93,7.68,93.24
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,62.97,58.90,59.83,59.35,...,857.55,61.71,41.78,29.17,214.91,307.62,394.70,19.93,9.96,91.61
4,Aaron Ness,19-20,ARI,D,24,292.63,49.44,51.47,49.69,49.55,...,276.12,10.93,10.53,-0.29,8.70,-2.64,-4.93,0.40,8.39,90.90


In [3]:
#Restrict Totals Data to GP,TOI,GF%,xGF%,GF,GA,xGF,xGA
skater_EV_totals = skater_EV_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G±','xG±']]
skater_EV_totals

,Player,Season,Team,Position,GP,TOI,G±,xG±
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40
...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,22.43,0.67
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.03,2.04
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,5.92,-8.20
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,-4.34,-0.41


In [4]:
#Rename columns
skater_EV_totals = skater_EV_totals.rename(columns = {'Position':'Position_EV','GP':'GP_EV','TOI':'TOI_EV','G+/-':'G_EV','xG+/-':'xG_EV'})
skater_EV_totals

,Player,Season,Team,Position_EV,GP_EV,TOI_EV,G±,xG±
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40
...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,22.43,0.67
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.03,2.04
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,5.92,-8.20
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,-4.34,-0.41


In [5]:
#Import PP On-Ice Rates Data
skater_PP_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/PP%20On-Ice%20Totals.csv')
skater_PP_totals_raw.head()

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,Aaron Ekblad,19-20,FLA,D,67,95.90,92.22,81.87,84.52,85.63,...,22.50,9.81,0.72,9.44,63.83,86.56,111.57,9.09,12.57,95.21
1,Aaron Ekblad,20-21,FLA,D,35,132.75,91.82,87.33,87.48,87.89,...,33.55,18.23,1.53,19.73,132.80,175.41,209.85,16.70,13.94,91.44
2,Aaron Ekblad,21-22,FLA,D,61,214.78,92.51,86.16,86.54,87.19,...,51.02,29.39,2.64,32.58,171.71,231.62,296.13,26.75,17.33,91.27
3,Adam Boqvist,19-20,CHI,D,41,87.02,88.20,81.06,84.35,86.69,...,19.11,8.91,0.91,6.41,53.41,75.83,105.34,8.00,10.62,93.92
4,Adam Boqvist,20-21,CHI,D,35,118.65,82.03,75.97,79.96,82.55,...,32.46,11.76,3.35,10.52,56.65,84.80,121.14,8.41,16.26,88.74


In [6]:
#Reduce PP On-Ice Totals Data
skater_PP_totals = skater_PP_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G±']]
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,G±
0,Aaron Ekblad,19-20,FLA,D,67,95.90,9.44
1,Aaron Ekblad,20-21,FLA,D,35,132.75,19.73
2,Aaron Ekblad,21-22,FLA,D,61,214.78,32.58
3,Adam Boqvist,19-20,CHI,D,41,87.02,6.41
4,Adam Boqvist,20-21,CHI,D,35,118.65,10.52
...,...,...,...,...,...,...,...
1278,Zach Werenski,19-20,CBJ,D,63,171.17,13.36
1279,Zach Werenski,20-21,CBJ,D,33,73.60,4.99
1280,Zach Werenski,21-22,CBJ,D,64,215.48,21.79
1281,Zack Kassian,19-20,EDM,R,58,23.18,0.19


In [7]:
#Add column for mean G+/- and xG+/-
G_mean = skater_PP_totals['G±'].mean()
#G_mean

In [8]:
#Add column for stats above average for G+/- and xG+/-
skater_PP_totals['GAA'] = skater_PP_totals['G±'] - G_mean
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,G±,GAA
0,Aaron Ekblad,19-20,FLA,D,67,95.90,9.44,-3.070109
1,Aaron Ekblad,20-21,FLA,D,35,132.75,19.73,7.219891
2,Aaron Ekblad,21-22,FLA,D,61,214.78,32.58,20.069891
3,Adam Boqvist,19-20,CHI,D,41,87.02,6.41,-6.100109
4,Adam Boqvist,20-21,CHI,D,35,118.65,10.52,-1.990109
...,...,...,...,...,...,...,...,...
1278,Zach Werenski,19-20,CBJ,D,63,171.17,13.36,0.849891
1279,Zach Werenski,20-21,CBJ,D,33,73.60,4.99,-7.520109
1280,Zach Werenski,21-22,CBJ,D,64,215.48,21.79,9.279891
1281,Zack Kassian,19-20,EDM,R,58,23.18,0.19,-12.320109


In [9]:
#Drop columns
skater_PP_totals = skater_PP_totals.drop(columns = ['G±'])
skater_PP_totals

,Player,Season,Team,Position,GP,TOI,GAA
0,Aaron Ekblad,19-20,FLA,D,67,95.90,-3.070109
1,Aaron Ekblad,20-21,FLA,D,35,132.75,7.219891
2,Aaron Ekblad,21-22,FLA,D,61,214.78,20.069891
3,Adam Boqvist,19-20,CHI,D,41,87.02,-6.100109
4,Adam Boqvist,20-21,CHI,D,35,118.65,-1.990109
...,...,...,...,...,...,...,...
1278,Zach Werenski,19-20,CBJ,D,63,171.17,0.849891
1279,Zach Werenski,20-21,CBJ,D,33,73.60,-7.520109
1280,Zach Werenski,21-22,CBJ,D,64,215.48,9.279891
1281,Zack Kassian,19-20,EDM,R,58,23.18,-12.320109


In [10]:
#Rename columns
skater_PP_totals = skater_PP_totals.rename(columns = {'Position':'Position_PP','GP':'GP_PP','TOI':'TOI_PP','GAA':'GAA_PP'})
skater_PP_totals

,Player,Season,Team,Position_PP,GP_PP,TOI_PP,GAA_PP
0,Aaron Ekblad,19-20,FLA,D,67,95.90,-3.070109
1,Aaron Ekblad,20-21,FLA,D,35,132.75,7.219891
2,Aaron Ekblad,21-22,FLA,D,61,214.78,20.069891
3,Adam Boqvist,19-20,CHI,D,41,87.02,-6.100109
4,Adam Boqvist,20-21,CHI,D,35,118.65,-1.990109
...,...,...,...,...,...,...,...
1278,Zach Werenski,19-20,CBJ,D,63,171.17,0.849891
1279,Zach Werenski,20-21,CBJ,D,33,73.60,-7.520109
1280,Zach Werenski,21-22,CBJ,D,64,215.48,9.279891
1281,Zack Kassian,19-20,EDM,R,58,23.18,-12.320109


In [11]:
#Import SH On-Ice Totals Data
skater_SH_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/SH%20On-Ice%20Totals.csv')
skater_SH_totals_raw.head()

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,Aaron Ekblad,19-20,FLA,D,65,134.87,7.45,22.51,21.41,19.08,...,227.77,3.86,19.19,-22.86,-98.13,-129.47,-174.07,-15.33,4.98,82.03
1,Aaron Ekblad,20-21,FLA,D,33,86.47,18.24,17.92,14.84,15.32,...,131.66,1.30,10.00,-7.00,-64.49,-93.88,-107.84,-8.70,11.15,89.08
2,Aaron Ekblad,21-22,FLA,D,61,166.83,25.76,19.05,17.45,16.22,...,257.22,4.56,19.13,-11.29,-122.07,-158.45,-207.42,-14.57,15.97,89.17
3,Adam Fox,20-21,NYR,D,54,139.75,18.44,20.18,17.25,15.49,...,233.65,2.48,17.19,-10.41,-87.56,-141.39,-190.84,-14.71,10.26,88.52
4,Adam Fox,21-22,NYR,D,78,163.50,17.42,11.66,11.33,9.96,...,276.05,1.65,20.36,-11.45,-136.27,-187.12,-245.50,-18.71,14.77,90.76


In [12]:
#Reduce SH On-Ice Totals Data
skater_SH_totals = skater_SH_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G±']]
skater_SH_totals

,Player,Season,Team,Position,GP,TOI,G±
0,Aaron Ekblad,19-20,FLA,D,65,134.87,-22.86
1,Aaron Ekblad,20-21,FLA,D,33,86.47,-7.00
2,Aaron Ekblad,21-22,FLA,D,61,166.83,-11.29
3,Adam Fox,20-21,NYR,D,54,139.75,-10.41
4,Adam Fox,21-22,NYR,D,78,163.50,-11.45
...,...,...,...,...,...,...,...
1279,Zdeno Chara,19-20,BOS,D,67,214.07,-21.23
1280,Zdeno Chara,20-21,WSH,D,55,145.72,-16.15
1281,Zdeno Chara,21-22,NYI,D,68,123.65,-5.59
1282,Zemgus Girgensons,19-20,BUF,C,68,147.87,-22.70


In [13]:
#Add column for mean G+/- and xG+/-
G_mean = skater_SH_totals['G±'].mean()
#G_mean

In [14]:
#Add column for stats above average for G+/- and xG+/-
skater_SH_totals['GAA'] = skater_SH_totals['G±'] - G_mean
skater_SH_totals

,Player,Season,Team,Position,GP,TOI,G±,GAA
0,Aaron Ekblad,19-20,FLA,D,65,134.87,-22.86,-13.197407
1,Aaron Ekblad,20-21,FLA,D,33,86.47,-7.00,2.662593
2,Aaron Ekblad,21-22,FLA,D,61,166.83,-11.29,-1.627407
3,Adam Fox,20-21,NYR,D,54,139.75,-10.41,-0.747407
4,Adam Fox,21-22,NYR,D,78,163.50,-11.45,-1.787407
...,...,...,...,...,...,...,...,...
1279,Zdeno Chara,19-20,BOS,D,67,214.07,-21.23,-11.567407
1280,Zdeno Chara,20-21,WSH,D,55,145.72,-16.15,-6.487407
1281,Zdeno Chara,21-22,NYI,D,68,123.65,-5.59,4.072593
1282,Zemgus Girgensons,19-20,BUF,C,68,147.87,-22.70,-13.037407


In [15]:
#Drop columns
skater_SH_totals = skater_SH_totals.drop(columns = ['G±'])
skater_SH_totals

,Player,Season,Team,Position,GP,TOI,GAA
0,Aaron Ekblad,19-20,FLA,D,65,134.87,-13.197407
1,Aaron Ekblad,20-21,FLA,D,33,86.47,2.662593
2,Aaron Ekblad,21-22,FLA,D,61,166.83,-1.627407
3,Adam Fox,20-21,NYR,D,54,139.75,-0.747407
4,Adam Fox,21-22,NYR,D,78,163.50,-1.787407
...,...,...,...,...,...,...,...
1279,Zdeno Chara,19-20,BOS,D,67,214.07,-11.567407
1280,Zdeno Chara,20-21,WSH,D,55,145.72,-6.487407
1281,Zdeno Chara,21-22,NYI,D,68,123.65,4.072593
1282,Zemgus Girgensons,19-20,BUF,C,68,147.87,-13.037407


In [16]:
#Rename columns
skater_SH_totals = skater_SH_totals.rename(columns = {'Position':'Position_SH','GP':'GP_SH','TOI':'TOI_SH','GAA':'GAA_SH'})
skater_SH_totals

,Player,Season,Team,Position_SH,GP_SH,TOI_SH,GAA_SH
0,Aaron Ekblad,19-20,FLA,D,65,134.87,-13.197407
1,Aaron Ekblad,20-21,FLA,D,33,86.47,2.662593
2,Aaron Ekblad,21-22,FLA,D,61,166.83,-1.627407
3,Adam Fox,20-21,NYR,D,54,139.75,-0.747407
4,Adam Fox,21-22,NYR,D,78,163.50,-1.787407
...,...,...,...,...,...,...,...
1279,Zdeno Chara,19-20,BOS,D,67,214.07,-11.567407
1280,Zdeno Chara,20-21,WSH,D,55,145.72,-6.487407
1281,Zdeno Chara,21-22,NYI,D,68,123.65,4.072593
1282,Zemgus Girgensons,19-20,BUF,C,68,147.87,-13.037407


In [17]:
#Import Player Box Stats
skater_box_totals_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Player%20Stats%20Box.csv')
skater_box_totals_raw.head()

,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points,...,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±
0,A.J. Greer,21-22,N.J,L,9,70.70,1,0,1,2,...,20,11,1,6,0,0,5,2,1,1
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13,41,...,34,56,14,10,0,0,-4,0,0,0
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6,22,...,24,46,14,8,1,1,-6,0,0,0
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29,57,...,62,54,13,9,0,0,-4,0,0,0
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0,1,...,17,49,0,5,0,0,5,0,0,0


In [18]:
#Reduce Data Stats
skater_box_totals = skater_box_totals_raw.loc[:,['Player','Season','Team','Position','GP','TOI','G','A1','A2','Points']]
#skater_box_totals.head()

In [19]:
#Look at positions to get correct F/D average points
forwards_table = skater_box_totals.loc[skater_box_totals.Position!= "D"]
defense_table = skater_box_totals.loc[skater_box_totals.Position== "D"]

#len(forwards_table)
#len(defense_table)
#print(len(forwards_table)+len(defense_table))

In [20]:
#Get Mean of Points
Points_mean = skater_box_totals['Points'].mean()
#print(Points_mean)

Forwards_mean = forwards_table['Points'].mean()
#print(Forwards_mean)

Defense_mean = defense_table['Points'].mean()
#print(Defensemen_mean)

In [21]:
#Get PAA
#skater_box_totals['PAA'] = skater_box_totals['Points'] - Points_mean
#skater_box_totals

#Get PAA for Fwds and Dmen and combine
pd.options.mode.chained_assignment = None
forwards_table['PAA'] = forwards_table['Points'] - Forwards_mean
#forwards_table.head(10)

defense_table['PAA'] = defense_table['Points'] - Defense_mean
#defensemen_table.head(10)

#Add dataframes back together and sort again
concat_frames = [forwards_table, defense_table]
skater_box_totals = pd.concat(concat_frames)
skater_box_totals = skater_box_totals.sort_index()
#skater_box_totals.head(10)

In [22]:
#Combine skater EV and PP stats
combined_player_df = pd.merge(skater_EV_totals, skater_PP_totals, how="outer", on=['Player','Season'])
combined_player_df.head()


,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G±,xG±,Team_y,Position_PP,GP_PP,TOI_PP,GAA_PP
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,NaN,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,FLA,D,67.0,95.90,-3.070109
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,FLA,D,35.0,132.75,7.219891
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,FLA,D,61.0,214.78,20.069891
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,NaN,NaN,NaN


In [23]:
#Drop duplicate columns
skater_EV_PP = combined_player_df.drop(columns = ['Team_y','Position_PP','GP_PP'])
skater_EV_PP

,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G±,xG±,TOI_PP,GAA_PP
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,22.43,0.67,NaN,NaN
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.03,2.04,NaN,NaN
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,5.92,-8.20,NaN,NaN
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,-4.34,-0.41,NaN,NaN


In [24]:
#Merge EV,PP,SH tables together
full_skater_totals = pd.merge(skater_EV_PP, skater_SH_totals, how="left", on=["Player", "Season"])
full_skater_totals.head(10)

,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G±,xG±,TOI_PP,GAA_PP,Team,Position_SH,GP_SH,TOI_SH,GAA_SH
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,FLA,D,65.0,134.87,-13.197407
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,FLA,D,33.0,86.47,2.662593
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,FLA,D,61.0,166.83,-1.627407
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Adam Boqvist,19-20,CHI,D,41,563.95,3.21,-1.85,87.02,-6.100109,NaN,NaN,NaN,NaN,NaN
6,Adam Boqvist,20-21,CHI,D,35,459.97,-2.34,-3.11,118.65,-1.990109,NaN,NaN,NaN,NaN,NaN
7,Adam Boqvist,21-22,CBJ,D,52,763.68,-3.32,-7.46,98.63,-4.540109,NaN,NaN,NaN,NaN,NaN
8,Adam Brooks,19-20,TOR,C,7,51.60,1.95,0.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Adam Brooks,20-21,TOR,C,11,110.38,3.13,0.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
#Clean up data
full_skater_totals = full_skater_totals.drop(columns = ['Team','Position_SH','GP_SH'])
full_skater_totals.head(10)

,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G±,xG±,TOI_PP,GAA_PP,TOI_SH,GAA_SH
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,NaN,NaN
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,NaN,NaN
5,Adam Boqvist,19-20,CHI,D,41,563.95,3.21,-1.85,87.02,-6.100109,NaN,NaN
6,Adam Boqvist,20-21,CHI,D,35,459.97,-2.34,-3.11,118.65,-1.990109,NaN,NaN
7,Adam Boqvist,21-22,CBJ,D,52,763.68,-3.32,-7.46,98.63,-4.540109,NaN,NaN
8,Adam Brooks,19-20,TOR,C,7,51.60,1.95,0.35,NaN,NaN,NaN,NaN
9,Adam Brooks,20-21,TOR,C,11,110.38,3.13,0.49,NaN,NaN,NaN,NaN


In [26]:
#Merge EV,PP,SH and Points tables together
full_skater_totals = pd.merge(full_skater_totals, skater_box_totals, how="left", on=["Player", "Season"])
full_skater_totals.head()

,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G±,xG±,TOI_PP,GAA_PP,...,GAA_SH,Team,Position,GP,TOI,G,A1,A2,Points,PAA
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,...,NaN,N.J,L,9,70.70,1,0,1,2,-22.333540
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,...,-13.197407,FLA,D,67,1537.92,5,23,13,41,25.988649
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,...,2.662593,FLA,D,35,878.18,11,5,6,22,6.988649
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,...,-1.627407,FLA,D,61,1519.70,15,13,29,57,41.988649
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,...,NaN,ARI,D,24,300.83,0,1,0,1,-14.011351


In [27]:
#Drop Repeated Columns
full_skater_totals = full_skater_totals.drop(columns = ['Team','Position','GP','G','A1','A2','Points'])
full_skater_totals.head(10)


,Player,Season,Team_x,Position_EV,GP_EV,TOI_EV,G±,xG±,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI,PAA
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,NaN,NaN,70.70,-22.333540
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407,1537.92,25.988649
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593,878.18,6.988649
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407,1519.70,41.988649
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,NaN,NaN,300.83,-14.011351
5,Adam Boqvist,19-20,CHI,D,41,563.95,3.21,-1.85,87.02,-6.100109,NaN,NaN,665.12,-2.011351
6,Adam Boqvist,20-21,CHI,D,35,459.97,-2.34,-3.11,118.65,-1.990109,NaN,NaN,594.65,0.988649
7,Adam Boqvist,21-22,CBJ,D,52,763.68,-3.32,-7.46,98.63,-4.540109,NaN,NaN,886.05,6.988649
8,Adam Brooks,19-20,TOR,C,7,51.60,1.95,0.35,NaN,NaN,NaN,NaN,54.97,-21.333540
9,Adam Brooks,20-21,TOR,C,11,110.38,3.13,0.49,NaN,NaN,NaN,NaN,117.45,-19.333540


In [28]:
#Rename Columns
full_skater_totals = full_skater_totals.rename(columns = {'Team_x':'Team','Position_EV':'Pos','GP_EV':'GP','TOI':'TOI_tot'})
full_skater_totals

,Player,Season,Team,Pos,GP,TOI_EV,G±,xG±,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI_tot,PAA
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,NaN,NaN,NaN,NaN,70.70,-22.333540
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407,1537.92,25.988649
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593,878.18,6.988649
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407,1519.70,41.988649
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,NaN,NaN,NaN,NaN,300.83,-14.011351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,22.43,0.67,NaN,NaN,214.07,-11.567407,1428.48,-1.011351
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.03,2.04,NaN,NaN,145.72,-6.487407,1006.27,-5.011351
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,5.92,-8.20,NaN,NaN,123.65,4.072593,1348.45,-1.011351
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,-4.34,-0.41,NaN,NaN,147.87,-13.037407,951.77,-5.333540


In [29]:
#Replace NaN with 0
full_skater_totals = full_skater_totals.replace(np.NaN, 0)
full_skater_totals

,Player,Season,Team,Pos,GP,TOI_EV,G±,xG±,TOI_PP,GAA_PP,TOI_SH,GAA_SH,TOI_tot,PAA
0,A.J. Greer,21-22,N.J,L,9,70.45,0.06,0.94,0.00,0.000000,0.00,0.000000,70.70,-22.333540
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,12.38,-0.18,95.90,-3.070109,134.87,-13.197407,1537.92,25.988649
2,Aaron Ekblad,20-21,FLA,D,35,630.15,6.70,4.93,132.75,7.219891,86.47,2.662593,878.18,6.988649
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,29.17,19.93,214.78,20.069891,166.83,-1.627407,1519.70,41.988649
4,Aaron Ness,19-20,ARI,D,24,292.63,-0.29,0.40,0.00,0.000000,0.00,0.000000,300.83,-14.011351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,22.43,0.67,0.00,0.000000,214.07,-11.567407,1428.48,-1.011351
2476,Zdeno Chara,20-21,WSH,D,55,817.42,2.03,2.04,0.00,0.000000,145.72,-6.487407,1006.27,-5.011351
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,5.92,-8.20,0.00,0.000000,123.65,4.072593,1348.45,-1.011351
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,-4.34,-0.41,0.00,0.000000,147.87,-13.037407,951.77,-5.333540


In [30]:
#Import Goalie Data
goalie_stats_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Goalie%20Stats.csv')
goalie_stats_raw.head()

,Player,Season,Team,Position,GP,TOI,GA,SA,FA,xGA,Sv%,FSv%,xFSv%,dFSv%,GSAA,GSAx
0,Aaron Dell,19-20,S.J,G,33,1834.23,91.06,994.37,1352.10,92.16,90.84,93.27,93.18,0.08,-1.20,1.10
1,Aaron Dell,20-21,N.J,G,7,319.17,21.45,153.29,210.23,11.91,86.01,89.80,94.33,-4.54,-7.35,-9.54
2,Aaron Dell,21-22,BUF,G,12,565.38,37.95,356.65,467.99,30.60,89.36,91.89,93.46,-1.57,-4.85,-7.35
3,Adam Huska,21-22,NYR,G,1,59.70,7.53,42.56,55.97,3.50,82.31,86.55,93.75,-7.20,-3.58,-4.03
4,Adam Werner,19-20,COL,G,2,87.83,4.85,54.22,65.83,3.76,91.05,92.63,94.29,-1.66,0.05,-1.09


In [31]:
#Reduce Goalie Stats
goalie_stats = goalie_stats_raw.loc[:,['Player','Season','Team','Position','GP','GSAx']]
goalie_stats.head()

,Player,Season,Team,Position,GP,GSAx
0,Aaron Dell,19-20,S.J,G,33,1.10
1,Aaron Dell,20-21,N.J,G,7,-9.54
2,Aaron Dell,21-22,BUF,G,12,-7.35
3,Adam Huska,21-22,NYR,G,1,-4.03
4,Adam Werner,19-20,COL,G,2,-1.09


In [32]:
#Add Active / Inactive Column to both data tables
full_skater_totals['A/I'] = 'Active'
goalie_stats['A/I'] = 'Active'

In [ ]:
#List of team names
#print(full_player_totals['Team'].unique())

In [ ]:
#Put Players on current teams

#NHL API info
#https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d
#https://github.com/dword4/nhlapi
#https://gitlab.com/dword4/nhlapi

#API JSON project example
#https://github.com/danmartin25/World_Weather_Analysis/blob/main/WeatherPy.ipynb

#Test all
#data = requests.get('https://statsapi.web.nhl.com/api/v1/configurations').json()
#data

#Different options
#data = requests.get('https://statsapi.web.nhl.com/api/v1/expands').json()
#data

#Empty Dictionary to Hold Rosters
#active_rosters = []

### NOTE: ONLY RUN REQUEST ONCE, THEN COMMENT OUT. INFO WILL STAY IN VARIABLE
#json_api_data = requests.get('https://statsapi.web.nhl.com/api/v1/teams?expand=team.roster').json()

#Variable where data is stored
#json_api_data

#Devils Roster (Index 0, change indext to 1, 2, etc for other teams rosters)
#json_api_data["teams"][0]

#player_name = json_api_data["teams"][0]
#player_team = 
#player_position = #code


#active_rosters.append{{"Player": player_name,
#                       "Team": player_team,
#                       "Postion": player_position}}




In [33]:
#Make a Data Table for skaters on each team
Avalanche_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'COL')]
Blackhawks_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'CHI')]
Blue_Jackets_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'CBJ')]
Blues_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'STL')]
Bruins_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'BOS')]
Canadiens_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'MTL')]
Canucks_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'VAN')]
Capitals_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'WSH')]
Coyotes_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'ARI')]
Devils_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'N.J')]
Ducks_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'ANA')]
Flames_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'CGY')]
Flyers_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'PHI')]
Hurricanes_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'CAR')]
Islanders_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'NYI')]
Jets_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'WPG')]
Kings_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'L.A')]
Knights_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'VGK')]
Kraken_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'SEA')]
Leafs_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'TOR')]
Lightning_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'T.B')]
Oilers_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'EDM')]
Panthers_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'FLA')]
Penguins_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'PIT')]
Predators_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'NSH')]
Rangers_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'NYR')]
Red_Wings_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'DET')]
Sabres_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'BUF')]
Senators_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'OTT')]
Sharks_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'S.J')]
Stars_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'DAL')]
Wild_skaters = full_skater_totals.loc[(full_skater_totals.Team == 'MIN')]
#Avalanche_skaters.head(10)

In [35]:
#Make a Data for each team of goalies
Avalanche_goalie = goalie_stats.loc[(goalie_stats.Team == 'COL')]
Blackhawks_goalie = goalie_stats.loc[(goalie_stats.Team == 'CHI')]
Blue_Jackets_goalie = goalie_stats.loc[(goalie_stats.Team == 'CBJ')]
Blues_goalie = goalie_stats.loc[(goalie_stats.Team == 'STL')]
Bruins_goalie = goalie_stats.loc[(goalie_stats.Team == 'BOS')]
Canadiens_goalie = goalie_stats.loc[(goalie_stats.Team == 'MTL')]
Canucks_goalie = goalie_stats.loc[(goalie_stats.Team == 'VAN')]
Capitals_goalie = goalie_stats.loc[(goalie_stats.Team == 'WSH')]
Coyotes_goalie = goalie_stats.loc[(goalie_stats.Team == 'ARI')]
Devils_goalie = goalie_stats.loc[(goalie_stats.Team == 'N.J')]
Ducks_goalie = goalie_stats.loc[(goalie_stats.Team == 'ANA')]
Flames_goalie = goalie_stats.loc[(goalie_stats.Team == 'CGY')]
Flyers_goalie = goalie_stats.loc[(goalie_stats.Team == 'PHI')]
Hurricanes_goalie = goalie_stats.loc[(goalie_stats.Team == 'CAR')]
Islanders_goalie = goalie_stats.loc[(goalie_stats.Team == 'NYI')]
Jets_goalie = goalie_stats.loc[(goalie_stats.Team == 'WPG')]
Kings_goalie = goalie_stats.loc[(goalie_stats.Team == 'L.A')]
Knights_goalie = goalie_stats.loc[(goalie_stats.Team == 'VGK')]
Kraken_goalie = goalie_stats.loc[(goalie_stats.Team == 'SEA')]
Leafs_goalie = goalie_stats.loc[(goalie_stats.Team == 'TOR')]
Lightning_goalie = goalie_stats.loc[(goalie_stats.Team == 'T.B')]
Oilers_goalie = goalie_stats.loc[(goalie_stats.Team == 'EDM')]
Panthers_goalie = goalie_stats.loc[(goalie_stats.Team == 'FLA')]
Penguins_goalie = goalie_stats.loc[(goalie_stats.Team == 'PIT')]
Predators_goalie = goalie_stats.loc[(goalie_stats.Team == 'NSH')]
Rangers_goalie = goalie_stats.loc[(goalie_stats.Team == 'NYR')]
Red_Wings_goalie = goalie_stats.loc[(goalie_stats.Team == 'DET')]
Sabres_goalie = goalie_stats.loc[(goalie_stats.Team == 'BUF')]
Senators_goalie = goalie_stats.loc[(goalie_stats.Team == 'OTT')]
Sharks_goalie = goalie_stats.loc[(goalie_stats.Team == 'S.J')]
Stars_goalie = goalie_stats.loc[(goalie_stats.Team == 'DAL')]
Wild_goalie = goalie_stats.loc[(goalie_stats.Team == 'MIN')]

In [36]:
Avalanche_goalie

,Player,Season,Team,Position,GP,GSAx,A/I
4,Adam Werner,19-20,COL,G,2,-1.09,Active
28,Antoine Bibeau,19-20,COL,G,2,-0.57,Active
93,Darcy Kuemper,21-22,COL,G,57,15.77,Active
117,Hunter Miska,20-21,COL,G,5,-9.33,Active
175,Justus Annunen,21-22,COL,G,2,-2.65,Active
231,Pavel Francouz,19-20,COL,G,34,2.68,Active
232,Pavel Francouz,21-22,COL,G,21,-0.10,Active
239,Philipp Grubauer,19-20,COL,G,36,-1.71,Active
240,Philipp Grubauer,20-21,COL,G,40,4.79,Active


**Stuff to Do Next - (have no more time, so quick thoughts that I have not checked, like the next one would be so easy to check if I had another 5 mins)**

In [ ]:
#Are these right? duplicate naming issues for "TOI_SH" and no PP?
full_skater_totals.columns

Index(['Player', 'Season', 'Team', 'Position', 'GP', 'TOI_EV', 'G_EV', 'xG_EV',
       'TOI_PP', 'GAA_PP', 'TOI_SH', 'GAA_SH', 'TOI_tot', 'Points', 'PAA',
       'A/I'],
      dtype='object')

Need to create a dataframe for each team. Use the loc function on player table first to get all the players for one team. Then do the same thing with goalies. We will probably need to keep them seperate tables since they do not have similar variables

Not necessary at this stage but eventually we are going to want to consolidate these code blocks. Ex, the code to create all of the team rosters should be in one block with no output. For now you can do 64 blocks (32 for skaters, 32 for goalies) and show output so we can see if there are any errors, but once we get past that stage we will comment out the output and put it all in one block. If we ever need to go back in check you just take out the '#' and run the code to see the output.

Need to add a column for 'status'. I am thinking the two states we use are 'active' and 'inactive' for the starting lineups. Will be 18 skaters and 1 goalie when we update daily but doesnt matter now. Should put this in before all of the loc functions so it shows up for the team dataframes. Note: will have to be added to both the full_skater_totals dataframe and the goalie_stats dataframe.

In [ ]:
#Import Skater Box Car Stats
skater_box = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/Player%20Stats%20Box.csv')
skater_box

,Player,Season,Team,Position,GP,TOI,G,A1,A2,Points,...,iHF,iHA,iPENT2,iPEND2,iPENT5,iPEND5,iPEN±,FOW,FOL,FO±
0,A.J. Greer,21-22,N.J,L,9,70.70,1,0,1,2,...,20,11,1,6,0,0,5,2,1,1
1,Aaron Ekblad,19-20,FLA,D,67,1537.92,5,23,13,41,...,34,56,14,10,0,0,-4,0,0,0
2,Aaron Ekblad,20-21,FLA,D,35,878.18,11,5,6,22,...,24,46,14,8,1,1,-6,0,0,0
3,Aaron Ekblad,21-22,FLA,D,61,1519.70,15,13,29,57,...,62,54,13,9,0,0,-4,0,0,0
4,Aaron Ness,19-20,ARI,D,24,300.83,0,1,0,1,...,17,49,0,5,0,0,5,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,Zdeno Chara,19-20,BOS,D,68,1428.48,5,2,7,14,...,78,67,20,10,4,4,-10,0,0,0
2493,Zdeno Chara,20-21,WSH,D,55,1006.27,2,4,4,10,...,88,33,12,4,2,2,-8,0,0,0
2494,Zdeno Chara,21-22,NYI,D,72,1348.45,2,8,4,14,...,125,46,20,6,5,5,-14,0,0,0
2495,Zemgus Girgensons,19-20,BUF,C,69,951.77,12,4,3,19,...,110,71,5,6,0,0,1,30,43,-13


In [ ]:
#Restrict Data Table to Player, Season, Team, G, A1, A2, iSH, iBLK, iPEN±, FO±
skater_box = skater_box.loc[:,['Player','Season','Team','Position','GP','G','A1','A2','iSF','iBLK','iPEN±','FO±']]
skater_box

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1
1,Aaron Ekblad,19-20,FLA,D,67,5,23,13,147,84,-4,0
2,Aaron Ekblad,20-21,FLA,D,35,11,5,6,101,27,-6,0
3,Aaron Ekblad,21-22,FLA,D,61,15,13,29,180,69,-4,0
4,Aaron Ness,19-20,ARI,D,24,0,1,0,11,20,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2492,Zdeno Chara,19-20,BOS,D,68,5,2,7,86,101,-10,0
2493,Zdeno Chara,20-21,WSH,D,55,2,4,4,67,67,-8,0
2494,Zdeno Chara,21-22,NYI,D,72,2,8,4,81,90,-14,0
2495,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13


In [ ]:
#Import EV Skater Data
skater_EV = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Player%20Data/EV%20On-Ice%20Totals.csv')
skater_EV

,Player,Season,Team,Position,GP,TOI,GF%,SF%,FF%,CF%,...,CA,xGF,xGA,G±,S±,F±,C±,xG±,Sh%,Sv%
0,A.J. Greer,21-22,N.J,L,9,70.45,50.50,57.90,57.33,56.89,...,54.34,3.16,2.22,0.06,10.82,14.68,17.36,0.94,7.59,89.76
1,Aaron Ekblad,19-20,FLA,D,67,1268.28,54.75,53.16,53.30,53.14,...,1070.09,52.60,52.78,12.38,84.53,114.87,143.58,-0.18,10.03,90.59
2,Aaron Ekblad,20-21,FLA,D,35,630.15,57.67,54.51,53.65,53.97,...,481.39,26.51,21.58,6.70,54.32,59.26,83.07,4.93,7.68,93.24
3,Aaron Ekblad,21-22,FLA,D,61,1091.47,62.97,58.90,59.83,59.35,...,857.55,61.71,41.78,29.17,214.91,307.62,394.70,19.93,9.96,91.61
4,Aaron Ness,19-20,ARI,D,24,292.63,49.44,51.47,49.69,49.55,...,276.12,10.93,10.53,-0.29,8.70,-2.64,-4.93,0.40,8.39,90.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,1171.20,62.20,49.10,48.67,48.25,...,1085.01,42.25,41.58,22.43,-21.33,-43.04,-73.23,0.67,9.85,94.23
2476,Zdeno Chara,20-21,WSH,D,55,817.42,51.49,49.37,49.02,50.39,...,706.99,28.61,26.57,2.03,-9.96,-21.06,11.25,2.04,9.04,91.70
2477,Zdeno Chara,21-22,NYI,D,72,1195.98,53.26,44.88,44.56,43.73,...,1246.03,46.96,55.16,5.92,-123.65,-180.44,-277.79,-8.20,8.92,93.63
2478,Zemgus Girgensons,19-20,BUF,C,69,785.45,45.67,47.49,49.21,49.34,...,621.61,25.45,25.86,-4.34,-35.51,-15.13,-16.25,-0.41,6.82,92.66


In [ ]:
#Restrict Data to Goal Differential and Corsi Differential
skater_EV = skater_EV.loc[:,['Player', 'Season', 'Position','G±', 'C±']]
skater_EV

,Player,Season,Position,G±,C±
0,A.J. Greer,21-22,L,0.06,17.36
1,Aaron Ekblad,19-20,D,12.38,143.58
2,Aaron Ekblad,20-21,D,6.70,83.07
3,Aaron Ekblad,21-22,D,29.17,394.70
4,Aaron Ness,19-20,D,-0.29,-4.93
...,...,...,...,...,...
2475,Zdeno Chara,19-20,D,22.43,-73.23
2476,Zdeno Chara,20-21,D,2.03,11.25
2477,Zdeno Chara,21-22,D,5.92,-277.79
2478,Zemgus Girgensons,19-20,C,-4.34,-16.25


In [ ]:
#Merge EV and box car stats together
Game_Score = pd.merge(skater_box, skater_EV)
Game_Score

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1,0.06,17.36
1,Aaron Ekblad,19-20,FLA,D,67,5,23,13,147,84,-4,0,12.38,143.58
2,Aaron Ekblad,20-21,FLA,D,35,11,5,6,101,27,-6,0,6.70,83.07
3,Aaron Ekblad,21-22,FLA,D,61,15,13,29,180,69,-4,0,29.17,394.70
4,Aaron Ness,19-20,ARI,D,24,0,1,0,11,20,5,0,-0.29,-4.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,5,2,7,86,101,-10,0,22.43,-73.23
2476,Zdeno Chara,20-21,WSH,D,55,2,4,4,67,67,-8,0,2.03,11.25
2477,Zdeno Chara,21-22,NYI,D,72,2,8,4,81,90,-14,0,5.92,-277.79
2478,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13,-4.34,-16.25


In [ ]:
#Add column for player's average game score
Game_Score['GS'] = ((Game_Score['G']*0.75) + (Game_Score['A1']*0.7) + (Game_Score['A2']*0.55) + (Game_Score['iSF']*0.075) + (Game_Score['iBLK']*0.05) + (Game_Score['iPEN±']*0.15) + (Game_Score['FO±']*0.01) + (Game_Score['C±']*0.05) + (Game_Score['G±']*0.15))/ Game_Score['GP']
Game_Score

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±,GS
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1,0.06,17.36,0.459667
1,Aaron Ekblad,19-20,FLA,D,67,5,23,13,147,84,-4,0,12.38,143.58,0.756134
2,Aaron Ekblad,20-21,FLA,D,35,11,5,6,101,27,-6,0,6.70,83.07,0.806671
3,Aaron Ekblad,21-22,FLA,D,61,15,13,29,180,69,-4,0,29.17,394.70,1.258369
4,Aaron Ness,19-20,ARI,D,24,0,1,0,11,20,5,0,-0.29,-4.93,0.124375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2475,Zdeno Chara,19-20,BOS,D,68,5,2,7,86,101,-10,0,22.43,-73.23,0.275044
2476,Zdeno Chara,20-21,WSH,D,55,2,4,4,67,67,-8,0,2.03,11.25,0.264400
2477,Zdeno Chara,21-22,NYI,D,72,2,8,4,81,90,-14,0,5.92,-277.79,0.066299
2478,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13,-4.34,-16.25,0.280891


In [ ]:
#Separate Data into forwards and defensemen
forwards_GS = Game_Score.loc[Game_Score.Position!="D"]
defense_GS = Game_Score.loc[Game_Score.Position=="D"]

In [ ]:
#Calculate Average Game Score for forwards and defensemen
avg_GS_forwards = forwards_GS['GS'].mean()
avg_GS_defense = defense_GS['GS'].mean()

In [ ]:
#Calculate GSVA for forwards
pd.options.mode.chained_assignment = None
forwards_GS['GSVA'] = forwards_GS['GS']-avg_GS_forwards
forwards_GS

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±,GS,GSVA
0,A.J. Greer,21-22,N.J,L,9,1,0,1,14,3,5,1,0.06,17.36,0.459667,0.014407
8,Adam Brooks,19-20,TOR,C,7,0,2,1,2,1,1,0,1.95,-2.68,0.351214,-0.094046
9,Adam Brooks,20-21,TOR,C,11,4,0,1,10,2,1,-3,3.13,3.73,0.470545,0.025285
10,Adam Brooks,21-22,MTL/VGK/WPG,C,25,2,1,0,14,6,0,-11,2.24,-29.75,0.091540,-0.353720
11,Adam Erne,19-20,DET,L,56,2,3,0,63,25,-6,7,-21.29,-138.97,-0.024946,-0.470206
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472,Zack MacEwen,20-21,VAN,C,34,1,0,1,31,9,-2,0,-0.68,-25.54,0.070471,-0.374789
2473,Zack MacEwen,21-22,PHI,C,75,3,3,3,82,32,-9,-3,-13.76,-221.36,-0.010160,-0.455420
2474,Zack Smith,19-20,CHI,C,50,4,5,2,43,18,-7,-14,-3.10,-79.20,0.122200,-0.323060
2478,Zemgus Girgensons,19-20,BUF,C,69,12,4,3,85,20,1,-13,-4.34,-16.25,0.280891,-0.164369


In [ ]:
forwards_GS.loc[forwards_GS.Player=='Auston Matthews']

,Player,Season,Team,Position,GP,G,A1,A2,iSF,iBLK,iPEN±,FO±,G±,C±,GS,GSVA
218,Auston Matthews,19-20,TOR,C,70,47,19,14,290,60,10,92,19.58,185.69,1.366307,0.921047
219,Auston Matthews,20-21,TOR,C,52,41,15,10,222,47,9,33,25.06,148.83,1.512125,1.066865
220,Auston Matthews,21-22,TOR,C,73,60,26,20,348,62,4,153,28.36,467.33,1.823979,1.378719


In [ ]:
avg_GS_forwards

0.4452599914524993

Things for next steps
1. Add goalie table.
2. Separate Players to teams.
3. Create matchups
4. Determine Log Loss for Teams
5.